## Scraping notebook

#### All the steps of scraping demographics data from www.zipdatamaps.com, is shown here.

#### And the cleaning of the data

In [66]:
from html_table_parser.parser import HTMLTableParser
from pprint import pprint
import urllib.request
import pandas as pd
import numpy as np
import stringcase

In [46]:
# zip codes needed
zip_list = np.array([14580, 14623, 14612, 14616, 14624, 14607, 14526, 14620, 14606,
       14618, 14621, 14615, 14609, 14514, 14472, 14619, 14559, 14617,
       14611, 14626, 14450, 14622, 14467, 14625, 14445, 14610]).astype(str)
      
zip_list

array(['14580', '14623', '14612', '14616', '14624', '14607', '14526',
       '14620', '14606', '14618', '14621', '14615', '14609', '14514',
       '14472', '14619', '14559', '14617', '14611', '14626', '14450',
       '14622', '14467', '14625', '14445', '14610'], dtype='<U11')

In [54]:
#Function to return a dataframe of zip data information
def get_table(zipcode):
    
    url = 'https://www.zipdatamaps.com/' + zipcode
    
 
    agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36\
    (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
  
    #making request to the website
    req = urllib.request.Request(url=url, headers={'User-Agent': agent})
    f = urllib.request.urlopen(req)

 
    # defining the html contents of a URL.
    xhtml = f.read().decode('utf-8')

    # HTMLTableParser object
    p = HTMLTableParser()

    # feeding the html contents in the
    # HTMLTableParser object
    p.feed(xhtml)

    #returning table as a dataframe
    return pd.DataFrame(p.tables[0])

In [140]:
def process_df(df,zipcode):
    #Turning the table from the website into a useable, dataframe.
    
    # Transposing dataframe
    df = df.T
    
    #Applying processing steps
    df.columns = [stringcase.alphanumcase(x) for x in df.iloc[0]]
    df.drop(index=0, inplace=True)
    df.reset_index(inplace=True, drop=True)
    # Adding zipcode column
    df['zipcode'] = zipcode
    
    return df

In [178]:
df_list = []

for x in zip_list:
    
    zip_table =  get_table(x)
    df_list.append(process_df(zip_table,x))

result = pd.concat(df_list)
result.reset_index(inplace=True, drop= True)

In [179]:
df_demographics = result[['CurrentPopulation', 'RacialMajority',
       'PublicSchoolRacialMajority', 'UnemploymentRate',
       'MedianHouseholdIncome', 'AverageAdjustedGrossIncome',
       'SchoolTestPerformance', 
       'Area','zipcode']]

In [181]:
cols = list(df_demographics.columns)
cols = [cols[-1]] + cols[0:-1]
df_demographics = df_demographics[cols]
df_demographics.head()

,zipcode,CurrentPopulation,RacialMajority,PublicSchoolRacialMajority,UnemploymentRate,MedianHouseholdIncome,AverageAdjustedGrossIncome,SchoolTestPerformance,Area
0,14580,50587,White 90.24%,White 87.1%,5.3%,$75618,$81510,Above Average,44 Sqm.
1,14623,27173,White 71.17%,White 56%,5.5%,$54283,$49130,Above Average,20 Sqm.
2,14612,34515,White 86.99%,White 69.2%,7.3%,$62148,$61510,Average,17 Sqm.
3,14616,28534,White 82.2%,White 67.2%,10.8%,$55438,$46270,Above Average,6 Sqm.
4,14624,36296,White 81.99%,White 66.6%,6.5%,$69312,$62360,Average,28 Sqm.


In [183]:
df_demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   zipcode                     26 non-null     object
 1   CurrentPopulation           26 non-null     object
 2   RacialMajority              26 non-null     object
 3   PublicSchoolRacialMajority  25 non-null     object
 4   UnemploymentRate            26 non-null     object
 5   MedianHouseholdIncome       26 non-null     object
 6   AverageAdjustedGrossIncome  26 non-null     object
 7   SchoolTestPerformance       26 non-null     object
 8   Area                        26 non-null     object
dtypes: object(9)
memory usage: 2.0+ KB
